# InfoGAN CeleA

[InfoGAN: Interpretable Representation Learning by Information Maximizing Generative Adversarial Nets](https://arxiv.org/abs/1606.03657).


In [1]:
#Import the libraries we will need.
import os, glob, cv2, math, sys
import tensorflow as tf
import numpy as np
from numpy.random import permutation
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
import tensorflow.contrib.slim as slim
import scipy.misc
import scipy
#from PIL import Image

np.random.seed(1)
#plt.ion()   # interactive mode
%matplotlib inline

load CeleA data

In [2]:
# http://docs.opencv.org/3.0-beta/doc/py_tutorials/py_gui/py_image_display/py_image_display.html
#img_rows, img_cols = 100, 100
#img_rows, img_cols = 64, 64
img_rows, img_cols = 32, 32

def get_im(path):

    #img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    img = cv2.imread(path)
    #img = plt.imread(path)
    #resized = img
    resized = cv2.resize(img, (img_cols, img_rows))

    return resized

def read_train_data_fullname(path):

    
    files = glob.glob(path)
    # shuffling
    filenames = permutation(files)
    
    return filenames

def read_train_data_fullname_lfw(path):

    root = path
    all_folders = os.listdir(root)
    path = []
    for afolder in all_folders:
        path.append(root+"/"+afolder+"/*.jpg")
    #print(path)
    files = []
    for apath in path:
        templist = glob.glob(apath)
        for afile in templist:
            files.append(afile)

    # shuffling
    filenames = permutation(files)
    
    return filenames

def read_train_data_batch(filenames, batchsize=5):
    
    end = min(len(filenames), batchsize)
    train_data = []
    
    for fl in filenames[:end]:
        flbase = os.path.basename(fl)
        img = get_im(fl)
        img = np.array(img, dtype=np.float32)
        
        #normalization
        #img -= np.mean(img)
        #img /= np.std(img)
        
        train_data.append(img)
    
    # list to np.array
    train_data = np.array(train_data, dtype=np.float32)

    # dataid, height, width, #channel -> dataid, #channel, height, width
    #train_data = train_data.transpose((0, 3, 1, 2))
    filenames = filenames[end:]
    
    return train_data, filenames

def read_train_data_mini_batch(filenames, startpoint, batchsize=5):
    
    train_data = []
    
    for fl in filenames[startpoint:startpoint+batchsize]:
        flbase = os.path.basename(fl)
        img = get_im(fl)
        img = np.array(img, dtype=np.float32)
        #print(type(img))        
        #normalization
        img -= np.mean(img)
        img /= np.std(img)
        #print(img.shape)
        
        train_data.append(img)
    
    # list to np.array
    train_data = np.array(train_data, dtype=np.float32)
    
    return train_data

#random mini-batch
def read_train_data_random_batch(filenames, batchsize=5):
    fullsize=len(filenames)
    #http://qiita.com/hamukazu/items/ec1b4659df00f0ce43b1
    idset = np.random.randint(0, high=fullsize, size=batchsize)
    #print(idset) 
    train_data = []
    
    for fid in idset:
        fl = filenames[fid]
        flbase = os.path.basename(fl)
        img = get_im(fl)
        #img[:,:,1] = 0
        #img[:,:,2] = 0
        img = np.array(img, dtype=np.float32)
        #img = np.reshape(img,[img.shape[0],img.shape[1],1])
        # normalization
        # https://stats.stackexchange.com/questions/185853/why-do-we-need-to-normalize-the-images-before-we-put-them-into-cnn
        img -= np.mean(img)
        img /= np.std(img)
        
        train_data.append(img)
    
    # list to np.array
    train_data = np.array(train_data, dtype=np.float32)

    # dataid, height, width, #channel -> dataid, #channel, height, width
    #train_data = train_data.transpose((0, 3, 1, 2))
    
    return train_data
# must be full path, ~/... not ok
# train_data_filenames = read_train_data_fullname("/home/cli/CeleA/Img/img_align_celeba/*.jpg")
#train_data,train_data_filenames = read_train_data_batch(train_data_filenames)

#random mini-batch
#train_data = read_train_data_random_batch(train_data_filenames)

In [3]:
#print(len(train_data),len(train_data_filenames))
#print(len(train_data[0][1]))

def imshow(inp, title=None):
    """Imshow for Tensor."""
    # need this transpose if there is transpose in read_train_data_xx calls
    #inp = inp.numpy().transpose((1, 2, 0))
    #inp = std * inp + mean
    #plt.imshow(inp,cmap="Purples",interpolation = 'bicubic')
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.01)  # pause a bit so that plots are updated

#train_data_filenames = read_train_data_fullname("/home/cli/CeleA/Img/img_align_celeba/*.jpg")
#train_data_filenames = read_train_data_fullname("/home/cli/CeleA/Img/img_align_celeba_png.7z/img_align_celeba_png/*.png")
#train_data_filenames = read_train_data_fullname_lfw("/home/cli/LFW/lfw2")
#print(len(train_data_filenames))
#print(train_data_filenames[3])
#train_data = read_train_data_mini_batch(train_data_filenames,0)  
#print(train_data[0].shape)
#print(train_data[43][:,:,0:2].shape)

#imshow(train_data[2][:,:,0])
#imshow(train_data[2][:,:,0])

### Helper Functions

In [4]:
#This function performns a leaky relu activation, which is needed for the discriminator network.
def lrelu(x, leak=0.2, name="lrelu"):
     with tf.variable_scope(name):
         f1 = 0.5 * (1 + leak)
         f2 = 0.5 * (1 - leak)
         return f1 * x + f2 * abs(x)
    
#The below functions are taken from carpdem20's implementation https://github.com/carpedm20/DCGAN-tensorflow
#They allow for saving sample images from the generator to follow progress
def save_images(images, size, image_path):
    return imsave(inverse_transform(images), size, image_path)

def imsave(images, size, path):
    return scipy.misc.imsave(path, merge_color(images, size))
    #return scipy.misc.imsave(path, merge(images, size))

def inverse_transform(images):
    return images
    #return (images+1.)/2.

def merge(images, size):
    h, w = images.shape[1], images.shape[2]
    img = np.zeros((h * size[0], w * size[1]))

    for idx, image in enumerate(images):
        i = idx % size[1]
        j = idx // size[1]
        img[j*h:j*h+h, i*w:i*w+w] = image

    return img

def merge_color(images, size):
    h, w, c = images.shape[1], images.shape[2],images.shape[3]
    img = np.zeros((h * size[0], w * size[1],c))

    for idx, image in enumerate(images):
        i = idx % size[1]
        j = idx // size[1]
        img[j*h:j*h+h, i*w:i*w+w, :] = image[:,:,:]

    return img


## Defining the Adversarial Networks

### Generator Network

The generator takes a vector of random numbers and transforms it into a 32x32 image. Each layer in the network involves a strided  transpose convolution, batch normalization, and rectified nonlinearity. Tensorflow's slim library allows us to easily define each of these layers.

In [5]:
# https://www.tensorflow.org/api_docs/python/tf/depth_to_space
# http://qiita.com/tadOne/items/48302a399dcad44c69c8   Tensorflow - padding = VALID/SAMEの違いについて
#     so 3 tf.depth_to_space(genX,2) gives 4x2^3 = 32
# 

def generator(z):
    
    zP = slim.fully_connected(z,4*4*448,normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.relu,scope='g_project',weights_initializer=initializer)
    zCon = tf.reshape(zP,[-1,4,4,448])
    
    gen1 = slim.convolution2d(\
        zCon,num_outputs=256,kernel_size=[4,4],\
        padding="SAME",normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.relu,scope='g_conv1', weights_initializer=initializer)
    gen1 = tf.depth_to_space(gen1,2)
    
    gen2 = slim.convolution2d(\
        gen1,num_outputs=128,kernel_size=[4,4],\
        padding="SAME",normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.relu,scope='g_conv2', weights_initializer=initializer)
    gen2 = tf.depth_to_space(gen2,2)
    
    gen3 = slim.convolution2d(\
        gen2,num_outputs=64,kernel_size=[4,4],\
        padding="SAME",normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.relu,scope='g_conv3', weights_initializer=initializer)
    gen3 = tf.depth_to_space(gen3,2)
    
    g_out = slim.convolution2d(\
        gen3,num_outputs=3,kernel_size=[32,32],padding="SAME",\
        biases_initializer=None,activation_fn=tf.nn.tanh,\
        scope='g_out', weights_initializer=initializer)
    
    return g_out

### Discriminator Network
The discriminator network takes as input a 32x32 image and transforms it into a single valued probability of being generated from real-world data. Again we use tf.slim to define the convolutional layers, batch normalization, and weight initialization.

In [6]:
def discriminator(bottom, cat_list,conts, reuse=False):
    
    dis1 = slim.convolution2d(bottom,64,[4,4],padding="SAME",\
        biases_initializer=None,activation_fn=lrelu,\
        reuse=reuse,scope='d_conv1',weights_initializer=initializer)
    dis1 = tf.space_to_depth(dis1,2)
    
    dis2 = slim.convolution2d(dis1,128,[4,4],padding="SAME",\
        normalizer_fn=slim.batch_norm,activation_fn=lrelu,\
        reuse=reuse,scope='d_conv2', weights_initializer=initializer)
    dis2 = tf.space_to_depth(dis2,2)
    
    dis3 = slim.convolution2d(dis2,256,[4,4],padding="SAME",\
        normalizer_fn=slim.batch_norm,activation_fn=lrelu,\
        reuse=reuse,scope='d_conv3',weights_initializer=initializer)
    dis3 = tf.space_to_depth(dis3,2)
        
    dis4 = slim.fully_connected(slim.flatten(dis3),1024,activation_fn=lrelu,\
        reuse=reuse,scope='d_fc1', weights_initializer=initializer)
        
    d_out = slim.fully_connected(dis4,1,activation_fn=tf.nn.sigmoid,\
        reuse=reuse,scope='d_out', weights_initializer=initializer)
    
    q_a = slim.fully_connected(dis4,128,normalizer_fn=slim.batch_norm,\
        reuse=reuse,scope='q_fc1', weights_initializer=initializer)
    
    
    ## Here we define the unique layers used for the q-network. The number of outputs depends on the number of 
    ## latent variables we choose to define.
    q_cat_outs = []
    for idx,var in enumerate(cat_list):
        q_outA = slim.fully_connected(q_a,var,activation_fn=tf.nn.softmax,\
            reuse=reuse,scope='q_out_cat_'+str(idx), weights_initializer=initializer)
        q_cat_outs.append(q_outA)
    
    q_cont_outs = None
    if conts > 0:
        q_cont_outs = slim.fully_connected(q_a,conts,activation_fn=tf.nn.tanh,\
            reuse=reuse,scope='q_out_cont_'+str(conts), weights_initializer=initializer)
    
    #print("d_out"+str(d_out))
    return d_out,q_cat_outs,q_cont_outs

### Connecting them together

In [7]:
# https://www.tensorflow.org/api_docs/python/tf/split
# https://www.tensorflow.org/api_docs/python/tf/one_hot
# https://www.tensorflow.org/api_docs/python/tf/concat
# https://www.tensorflow.org/api_docs/python/tf/reduce_sum
# https://www.tensorflow.org/api_docs/python/tf/reduce_mean
# https://www.tensorflow.org/api_docs/python/tf/trainable_variables
# https://www.tensorflow.org/api_docs/python/tf/train/AdamOptimizer
# https://www.tensorflow.org/api_docs/python/tf/contrib/layers/batch_norm
# https://deepage.net/deep_learning/2016/10/26/batch_normalization.html
# z_lat: one_hot_size + z_size + number_continuous = 10+64+2=76
# g_loss def is interesting, my understanding: 
#        if Dg is the probablity to be told as feak data, then 1-Dg is the probabily of suceessfully cheating, 
#        so we cal KL(Dg/(1-Dg)), and readuce_mean works as sampling proceduce
# 

tf.reset_default_graph()


z_size = 128 #Size of initial z vector used for generator.

# Define latent variables.
#categorical_list = [10]*10 # Each entry in this list defines a categorical variable of a specific size.
categorical_list = [10] # Each entry in this list defines a categorical variable of a specific size.
# categorical_list = [10,10] # Each entry in this list defines a categorical variable of a specific size.
number_continuous = 1 # The number of continous variables.

#This initializaer is used to initialize all the weights of the network.
initializer = tf.truncated_normal_initializer(stddev=0.02)

#These placeholders are used for input into the generator and discriminator, respectively.
z_in = tf.placeholder(shape=[None,z_size],dtype=tf.float32) #Random vector
real_in = tf.placeholder(shape=[None,32,32,3],dtype=tf.float32) #Real images

#These placeholders load the latent variables.
latent_cat_in = tf.placeholder(shape=[None,len(categorical_list)],dtype=tf.int32)
#print("latent_cat_in:", latent_cat_in)
latent_cat_list = tf.split(latent_cat_in,len(categorical_list),1)
#print("latent_cat_list: ",latent_cat_list)
if number_continuous>0:
    latent_cont_in = tf.placeholder(shape=[None,number_continuous],dtype=tf.float32)

oh_list = []
for idx,var in enumerate(categorical_list):
    latent_oh = tf.one_hot(tf.reshape(latent_cat_list[idx],[-1]),var)
    #print(latent_cat_list[idx])
    #print(latent_oh),  woundn't print anything in sess.run()
    oh_list.append(latent_oh)

#Concatenate all c and z variables.
z_lats = oh_list[:]
#print("1st z_lats: ", z_lats )
z_lats.append(z_in)
#print("2nd z_lats: ", z_lats )
if number_continuous>0:
    z_lats.append(latent_cont_in)
#print("3rd z_lats: ", z_lats )
z_lat = tf.concat(z_lats,1)
#print("z_lat: ", z_lat )

Gz = generator(z_lat) #Generates images from random z vectors
#print (Gz.shape)
Dx,_,_ = discriminator(real_in,categorical_list,number_continuous) #Produces probabilities for real images
Dg,QgCat,QgCont = discriminator(Gz,categorical_list,number_continuous,reuse=True) #Produces probabilities for generator images

#These functions together define the optimization objective of the GAN.
d_loss = -tf.reduce_mean(tf.log(Dx) + tf.log(1.-Dg)) #This optimizes the discriminator.
#g_loss = -tf.reduce_mean(tf.log((Dg/(1.-Dg)))) #KL Divergence optimizer
g_loss = -tf.reduce_mean(tf.log(Dg)) 

#Combine losses for each of the categorical variables.
cat_losses = []
for idx,latent_var in enumerate(oh_list):
    #print ("latent_var: ", latent_var)
    #print ("tf.log(QgCat[idx]): ",tf.log(QgCat[idx]))
    cat_loss = -tf.reduce_sum(latent_var*tf.log(QgCat[idx]),axis=1)
    cat_losses.append(cat_loss)
    
#Combine losses for each of the continous variables.
if number_continuous > 0:
    q_cont_loss = tf.reduce_sum(0.5 * tf.square(latent_cont_in - QgCont),axis=1)
else:
    q_cont_loss = tf.constant(0.0)

q_cont_loss = tf.reduce_mean(q_cont_loss)
q_cat_loss = tf.reduce_mean(cat_losses)
q_loss = tf.add(q_cat_loss,q_cont_loss)
tvars = tf.trainable_variables()
#print (len(tvars))
#for i in tvars:
#    print(i)

#The below code is responsible for applying gradient descent to update the GAN.
trainerD = tf.train.AdamOptimizer(learning_rate=0.0002,beta1=0.5)
trainerG = tf.train.AdamOptimizer(learning_rate=0.0002,beta1=0.9)
trainerQ = tf.train.AdamOptimizer(learning_rate=0.0002,beta1=0.5)
#
d_grads = trainerD.compute_gradients(d_loss,tvars[9:-2-((number_continuous>0)*2)-(len(categorical_list)*2)]) #Only update the weights for the discriminator network.
g_grads = trainerG.compute_gradients(g_loss, tvars[0:9]) #Only update the weights for the generator network.
q_grads = trainerQ.compute_gradients(q_loss, tvars) 

update_D = trainerD.apply_gradients(d_grads)
update_G = trainerG.apply_gradients(g_grads)
update_Q = trainerQ.apply_gradients(q_grads)

## Training the network
Now that we have fully defined our network, it is time to train it!

In [8]:
# on at52 (GTX1080), 15mins/10000 epochs , 5000000 is about 12.5 hrs　 
# https://stackoverflow.com/questions/19349410/how-to-pad-with-zeros-a-tensor-along-some-axis-python
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.pad.html
# blow up after 81800
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.reshape.html
# https://www.tensorflow.org/api_docs/python/tf/Session#run
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.hstack.html
c_val = 10

batch_size = 64 #Size of image batch to apply at each iteration.
#train_data_filenames = read_train_data_fullname("/home/cli/CeleA/Img/img_align_celeba_png.7z/img_align_celeba_png/*.png")
train_data_filenames = read_train_data_fullname_lfw("/home/cli/LFW/lfw2")

#iterations = 500000 #Total number of iterations to use.
iterations = 1000 #Total number of iterations to use.
sample_directory = './figsTut' #Directory to save sample images from generator in.
model_directory = './models' #Directory to save trained model to.

init = tf.global_variables_initializer()
saver = tf.train.Saver()
with tf.Session() as sess:  
    sess.run(init)
    for i in range(iterations):
        train_data_filenames=permutation(train_data_filenames) # mini-batch
        data_left = len(train_data_filenames)
        batch_counter = 0
        while data_left>0:
            batch_size_to_train = min(batch_size, data_left)          
            
            zs = np.random.uniform(-1.0,1.0,size=[batch_size_to_train,z_size]).astype(np.float32) #Generate a random z batch
            #print("zs shape:",zs.shape)
            
            #lcat = np.random.randint(0,10,[batch_size,len(categorical_list)]) #Generate random c batch
            lcat = np.random.randint(0,c_val,[batch_size_to_train,len(categorical_list)]) #Generate random c batch
            
            lcont = np.random.uniform(-1,1,[batch_size_to_train,number_continuous]) #

            #xs = read_train_data_random_batch(train_data_filenames, batchsize=batch_size_to_train)
            xs = read_train_data_mini_batch(train_data_filenames, batch_counter*batch_size, batch_size_to_train)
            
            _,dLoss = sess.run([update_D,d_loss],feed_dict={z_in:zs,real_in:xs,latent_cat_in:lcat,latent_cont_in:lcont}) #Update the discriminator
            _,gLoss = sess.run([update_G,g_loss],feed_dict={z_in:zs,latent_cat_in:lcat,latent_cont_in:lcont}) #Update the generator, twice for good measure.
            _,qLoss,qK,qC = sess.run([update_Q,q_loss,q_cont_loss,q_cat_loss],feed_dict={z_in:zs,latent_cat_in:lcat,latent_cont_in:lcont}) #Update to optimize mutual information.

            data_left = data_left - batch_size_to_train
            batch_counter +=1
            if batch_counter%1 == 0 or data_left == 0:
                z_sample = np.random.uniform(-1.0,1.0,size=[c_val*c_val,z_size]).astype(np.float32)
                lcat_sample = np.reshape(np.array([e for e in range(c_val) for _ in range(c_val)]),[c_val*c_val,1])
                a = np.reshape(np.array([[(e/4.5 - 1.)] for e in range(c_val) for _ in range(c_val)]),[c_val,c_val]).T
                b = np.reshape(a,[c_val*c_val,1])
                lcont_sample = b
                samples = sess.run(Gz,feed_dict={z_in:z_sample,latent_cat_in:lcat_sample,latent_cont_in:lcont_sample})
                if not os.path.exists(sample_directory):
                    os.makedirs(sample_directory)
                save_images(np.reshape(samples[0:100],[100,32,32,3]),[10,10],sample_directory+'/fig'\
                            +str(i)+'_'+str(batch_counter)+'.png')
                
                print ("epoch:"+str(i)+" batch_done:"+str(batch_counter) \
                       +" Gen Loss: " + str(gLoss) + " Disc Loss: " + str(dLoss) + " Q Losses: " + str([qK,qC]))
            
             
        """
        if i % 100 == 0:
            print ("Gen Loss: " + str(gLoss) + " Disc Loss: " + str(dLoss) + " Q Losses: " + str([qK,qC]))
            #z_sample = np.random.uniform(-1.0,1.0,size=[100,z_size]).astype(np.float32) #Generate another z batch
            z_sample = np.random.uniform(-1.0,1.0,size=[c_val*c_val,z_size]).astype(np.float32) #Generate another z batch
            #lcat_sample = np.reshape(np.array([e for e in range(10) for _ in range(10)]),[100,1])
            lcat_sample = np.reshape(np.array([e for e in range(c_val) for _ in range(c_val)]),[c_val*c_val,1])
            latent_fixed = np.ones((c_val*c_val,1))
            lcat_sample = np.hstack([latent_fixed,lcat_sample])
            
            #a = a = np.reshape(np.array([[(e/4.5 - 1.)] for e in range(10) for _ in range(10)]),[10,10]).T
            a = a = np.reshape(np.array([[(e/4.5 - 1.)] for e in range(c_val) for _ in range(c_val)]),[c_val,c_val]).T
            #b = np.reshape(a,[100,1])
            b = np.reshape(a,[c_val*c_val,1])
            c = np.zeros_like(b)
            lcont_sample = np.hstack([b,c])
            #
            samples = sess.run(Gz,feed_dict={z_in:z_sample,latent_cat_in:lcat_sample,latent_cont_in:lcont_sample}) #Use new z to get sample images from generator.
            if not os.path.exists(sample_directory):
                os.makedirs(sample_directory)
            #Save sample generator images for viewing training progress.
            #save_images(np.reshape(samples[0:100],[100,32,32]),[10,10],sample_directory+'/fig'+str(i)+'.png')
            save_images(np.reshape(samples[0:c_val*c_val],[c_val*c_val,32,32]),[c_val,c_val],sample_directory+'/fig'+str(i)+'.png')
        """
        
        if i % 10 == 0 and i != 0:
            if not os.path.exists(model_directory):
                os.makedirs(model_directory)
            saver.save(sess,model_directory+'/model-'+str(i)+'.cptk')
            print ("Saved Model on ", i)


epoch:0 batch_done:1 Gen Loss: 10.8093 Disc Loss: 1.50684 Q Losses: [0.21452199, 2.2875094]
epoch:0 batch_done:2 Gen Loss: 1.58906 Disc Loss: 1.13432 Q Losses: [0.16069031, 2.2909784]
epoch:0 batch_done:3 Gen Loss: 6.36265 Disc Loss: 4.44809 Q Losses: [0.15094638, 2.3174233]
epoch:0 batch_done:4 Gen Loss: 8.99172 Disc Loss: 0.0846714 Q Losses: [0.14221787, 2.2839708]
epoch:0 batch_done:5 Gen Loss: 7.67117 Disc Loss: 0.267323 Q Losses: [0.12819034, 2.2600918]
epoch:0 batch_done:6 Gen Loss: 4.51652 Disc Loss: 0.18706 Q Losses: [0.1161695, 2.2617178]
epoch:0 batch_done:7 Gen Loss: 11.6681 Disc Loss: 0.455332 Q Losses: [0.10314247, 2.2648273]
epoch:0 batch_done:8 Gen Loss: 13.2076 Disc Loss: 0.148518 Q Losses: [0.086699218, 2.2257361]
epoch:0 batch_done:9 Gen Loss: 10.7992 Disc Loss: 0.12668 Q Losses: [0.086579859, 2.2222574]
epoch:0 batch_done:10 Gen Loss: 6.70204 Disc Loss: 0.0171134 Q Losses: [0.1168533, 2.2004037]
epoch:0 batch_done:11 Gen Loss: 9.91904 Disc Loss: 0.11805 Q Losses: [0.

epoch:0 batch_done:87 Gen Loss: 9.55773 Disc Loss: 0.00733913 Q Losses: [0.068099394, 0.97649074]
epoch:0 batch_done:88 Gen Loss: 7.09193 Disc Loss: 0.00732735 Q Losses: [0.054263115, 0.89886761]
epoch:0 batch_done:89 Gen Loss: 16.1441 Disc Loss: 0.0247117 Q Losses: [0.059781365, 0.92551726]
epoch:0 batch_done:90 Gen Loss: 17.7336 Disc Loss: 0.00091038 Q Losses: [0.057572167, 0.92443705]
epoch:0 batch_done:91 Gen Loss: 15.6753 Disc Loss: 0.00349718 Q Losses: [0.065654255, 0.92350382]
epoch:0 batch_done:92 Gen Loss: 11.7957 Disc Loss: 0.00794005 Q Losses: [0.061219182, 0.95430517]
epoch:0 batch_done:93 Gen Loss: 10.2037 Disc Loss: 0.00331589 Q Losses: [0.062206741, 0.84793365]
epoch:0 batch_done:94 Gen Loss: 8.29435 Disc Loss: 0.00143072 Q Losses: [0.054034472, 0.84798622]
epoch:0 batch_done:95 Gen Loss: 8.24482 Disc Loss: 0.00743813 Q Losses: [0.043041974, 0.89094514]
epoch:0 batch_done:96 Gen Loss: 12.4938 Disc Loss: 0.0142093 Q Losses: [0.068678454, 0.86815917]
epoch:0 batch_done:97 

epoch:0 batch_done:171 Gen Loss: 8.51881 Disc Loss: 0.000766772 Q Losses: [0.036033295, 0.38278079]
epoch:0 batch_done:172 Gen Loss: 8.48498 Disc Loss: 0.00131874 Q Losses: [0.036467992, 0.39346576]
epoch:0 batch_done:173 Gen Loss: 9.8202 Disc Loss: 0.00312103 Q Losses: [0.033531755, 0.39052206]
epoch:0 batch_done:174 Gen Loss: 9.80705 Disc Loss: 0.00142974 Q Losses: [0.044168826, 0.39599335]
epoch:0 batch_done:175 Gen Loss: 8.97582 Disc Loss: 0.00167022 Q Losses: [0.037259065, 0.3826679]
epoch:0 batch_done:176 Gen Loss: 8.38162 Disc Loss: 0.00288368 Q Losses: [0.050628573, 0.35419488]
epoch:0 batch_done:177 Gen Loss: 5.96773 Disc Loss: 0.0184207 Q Losses: [0.029275533, 0.38465458]
epoch:0 batch_done:178 Gen Loss: 35.9029 Disc Loss: 0.0381076 Q Losses: [0.033647586, 0.40425986]
epoch:0 batch_done:179 Gen Loss: 28.8039 Disc Loss: 0.175909 Q Losses: [0.038307779, 0.38645226]
epoch:0 batch_done:180 Gen Loss: 22.1451 Disc Loss: 3.35276e-08 Q Losses: [0.04002618, 0.4037385]
epoch:0 batch_do

epoch:1 batch_done:48 Gen Loss: 9.4298 Disc Loss: 0.000611601 Q Losses: [0.030157564, 0.23599227]
epoch:1 batch_done:49 Gen Loss: 9.40932 Disc Loss: 0.000425807 Q Losses: [0.035700642, 0.24148746]
epoch:1 batch_done:50 Gen Loss: 9.13361 Disc Loss: 0.000841045 Q Losses: [0.026576821, 0.22764781]
epoch:1 batch_done:51 Gen Loss: 10.4615 Disc Loss: 0.00228071 Q Losses: [0.021198526, 0.21071537]
epoch:1 batch_done:52 Gen Loss: 10.535 Disc Loss: 0.000483788 Q Losses: [0.03012019, 0.2142621]
epoch:1 batch_done:53 Gen Loss: 9.99862 Disc Loss: 0.000285629 Q Losses: [0.029734172, 0.20253363]
epoch:1 batch_done:54 Gen Loss: 8.57687 Disc Loss: 0.00507256 Q Losses: [0.022817694, 0.21704213]
epoch:1 batch_done:55 Gen Loss: 11.9244 Disc Loss: 0.00403203 Q Losses: [0.016613595, 0.21390724]
epoch:1 batch_done:56 Gen Loss: 11.8545 Disc Loss: 0.00258152 Q Losses: [0.025919404, 0.21779111]
epoch:1 batch_done:57 Gen Loss: 11.5118 Disc Loss: 5.45525e-05 Q Losses: [0.039278317, 0.21777847]
epoch:1 batch_done

epoch:1 batch_done:134 Gen Loss: nan Disc Loss: nan Q Losses: [0.18191117, 2.3015685]
epoch:1 batch_done:135 Gen Loss: nan Disc Loss: nan Q Losses: [0.15215217, 2.3019886]
epoch:1 batch_done:136 Gen Loss: nan Disc Loss: nan Q Losses: [0.17277305, 2.3028955]
epoch:1 batch_done:137 Gen Loss: nan Disc Loss: nan Q Losses: [0.1389202, 2.3024936]
epoch:1 batch_done:138 Gen Loss: nan Disc Loss: nan Q Losses: [0.15831023, 2.3017516]
epoch:1 batch_done:139 Gen Loss: nan Disc Loss: nan Q Losses: [0.16180117, 2.3036289]
epoch:1 batch_done:140 Gen Loss: nan Disc Loss: nan Q Losses: [0.19563991, 2.3031483]
epoch:1 batch_done:141 Gen Loss: nan Disc Loss: nan Q Losses: [0.16302338, 2.3041024]
epoch:1 batch_done:142 Gen Loss: nan Disc Loss: nan Q Losses: [0.15476435, 2.3032959]
epoch:1 batch_done:143 Gen Loss: nan Disc Loss: nan Q Losses: [0.16259921, 2.3016219]
epoch:1 batch_done:144 Gen Loss: nan Disc Loss: nan Q Losses: [0.12351868, 2.3006711]
epoch:1 batch_done:145 Gen Loss: nan Disc Loss: nan Q L

epoch:2 batch_done:23 Gen Loss: nan Disc Loss: nan Q Losses: [0.19516993, 2.3030441]
epoch:2 batch_done:24 Gen Loss: nan Disc Loss: nan Q Losses: [0.18309525, 2.3023591]
epoch:2 batch_done:25 Gen Loss: nan Disc Loss: nan Q Losses: [0.15322037, 2.3028216]
epoch:2 batch_done:26 Gen Loss: nan Disc Loss: nan Q Losses: [0.16256255, 2.303494]
epoch:2 batch_done:27 Gen Loss: nan Disc Loss: nan Q Losses: [0.16095647, 2.3019819]
epoch:2 batch_done:28 Gen Loss: nan Disc Loss: nan Q Losses: [0.15136617, 2.3029158]
epoch:2 batch_done:29 Gen Loss: nan Disc Loss: nan Q Losses: [0.15625297, 2.3022563]


KeyboardInterrupt: 

## Using a trained network
Once we have a trained model saved, we may want to use it to generate new images, and explore the representation it has learned.

In [ ]:
# http://qiita.com/TokyoMickey/items/f6a9251f5a59120e39f8
"""
sample_directory = './figsTut' #Directory to save sample images from generator in.
model_directory = './models' #Directory to load trained model from.

#init = tf.initialize_all_variables()
c_val = 10

saver = tf.train.Saver()
with tf.Session() as sess:  
    sess.run(tf.global_variables_initializer())
    #sess.run(init)
    #Reload the model.
    print ('Loading Model...')
    ckpt = tf.train.get_checkpoint_state(model_directory)
    saver.restore(sess,ckpt.model_checkpoint_path)
    
    #z_sample = np.random.uniform(-1.0,1.0,size=[100,z_size]).astype(np.float32) #Generate another z batch
    z_sample = np.random.uniform(-1.0,1.0,size=[c_val*c_val,z_size]).astype(np.float32) #Generate another z batch
    #lcat_sample = np.reshape(np.array([e for e in range(10) for _ in range(10)]),[100,1])
    #lcat_sample = np.reshape(np.array([np.random.randint(10) for e in range(10) for _ in range(10)]),[100,1])
    lcat_sample = np.reshape(np.array([e for e in range(c_val) for _ in range(c_val)]),[c_val*c_val,1])
    #print(np.array([np.random.randint(10) for e in range(10) for _ in range(10)]))
    #latent_fixed = np.ones((c_val*c_val,1))*50
    latent_fixed = np.zeros((c_val*c_val,1))
    #lcat_sample = np.hstack([latent_fixed,lcat_sample])
    # good shape
    lcat_sample = np.hstack([lcat_sample,latent_fixed])
            
    #a = a = np.reshape(np.array([[(e/4.5 - 1.)] for e in range(10) for _ in range(10)]),[10,10]).T
    a = a = np.reshape(np.array([[(e*5/4.5 - 5.)] for e in range(c_val) for _ in range(c_val)]),[c_val,c_val]).T
    #a = a = np.ones((c_val*c_val,1))*-0.5
    #a = a = np.reshape(np.array([[(e*5/4.5 - 5.)] for e in range(c_val) for _ in range(c_val)]),[c_val,c_val]).T
    #b = np.reshape(a,[100,1])
    b = np.reshape(a,[c_val*c_val,1])
    #c = np.zeros_like(b)
    #c = np.zeros_like(b)
    c = np.zeros_like(b)
    #c = np.zeros_like(b)+8
    #angle
    lcont_sample = np.hstack([b,c])
    # width
    #lcont_sample = np.hstack([c,b])
    
    samples = sess.run(Gz,feed_dict={z_in:z_sample,latent_cat_in:lcat_sample,latent_cont_in:lcont_sample}) #Use new z to get sample images from generator.
    if not os.path.exists(sample_directory):
        os.makedirs(sample_directory)
    #Save sample generator images for viewing training progress.
    #save_images(np.reshape(samples[0:100],[100,32,32]),[10,10],sample_directory+'/fig_test'+'.png')
    #save_images(np.reshape(samples[0:100],[100,32,32]),[10,10],sample_directory+'/fig_test_4'+'.png')
    save_images(np.reshape(samples[0:c_val*c_val],[c_val*c_val,32,32]),[c_val,c_val],sample_directory+'/fig_test_13'+'.png')
""" 